In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e11/sample_submission.csv
/kaggle/input/playground-series-s5e11/train.csv
/kaggle/input/playground-series-s5e11/test.csv


In [2]:
import pandas as pd
import numpy as np

# Import the data ---

data_path = "/kaggle/input/playground-series-s5e11/"

train_df = pd.read_csv(data_path + "train.csv")
test_df = pd.read_csv(data_path + "test.csv")
sample_sub_df = pd.read_csv(data_path + "sample_submission.csv")

# Tip: Always drop the 'id' from the training data, as it's not a feature.
# We'll save the test 'id' for the submission file.
test_ids = test_df['id']
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)

# Now, you can start exploring!
print(train_df.head())
train_df.info()

   annual_income  debt_to_income_ratio  credit_score  loan_amount  \
0       29367.99                 0.084           736      2528.42   
1       22108.02                 0.166           636      4593.10   
2       49566.20                 0.097           694     17005.15   
3       46858.25                 0.065           533      4682.48   
4       25496.70                 0.053           665     12184.43   

   interest_rate  gender marital_status education_level employment_status  \
0          13.67  Female         Single     High School     Self-employed   
1          12.92    Male        Married        Master's          Employed   
2           9.76    Male         Single     High School          Employed   
3          16.10  Female         Single     High School          Employed   
4          10.21    Male        Married     High School          Employed   

         loan_purpose grade_subgrade  loan_paid_back  
0               Other             C3             1.0  
1  Debt cons

Step 2: Explore the Data (EDA)

In [3]:
# Check for missing values (train_df.info() also does this)
print(train_df.isnull().sum())

# Look at the balance of your target variable
print(train_df['loan_paid_back'].value_counts(normalize=True))

# Get stats on number columns
print(train_df.describe())

annual_income           0
debt_to_income_ratio    0
credit_score            0
loan_amount             0
interest_rate           0
gender                  0
marital_status          0
education_level         0
employment_status       0
loan_purpose            0
grade_subgrade          0
loan_paid_back          0
dtype: int64
loan_paid_back
1.0    0.79882
0.0    0.20118
Name: proportion, dtype: float64
       annual_income  debt_to_income_ratio   credit_score    loan_amount  \
count  593994.000000         593994.000000  593994.000000  593994.000000   
mean    48212.202976              0.120696     680.916009   15020.297629   
std     26711.942078              0.068573      55.424956    6926.530568   
min      6002.430000              0.011000     395.000000     500.090000   
25%     27934.400000              0.072000     646.000000   10279.620000   
50%     46557.680000              0.096000     682.000000   15000.220000   
75%     60981.320000              0.156000     719.000000   18858

Step 3: Preprocess the Data

In [4]:
# --- 2. Convert Text to Numbers (One-Hot Encoding) ---
# pd.get_dummies() will automatically find and convert all text columns
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

# --- 3. Align Columns ---
# This ensures both train and test have the exact same columns
# (in case one was missing a category)
train_cols = train_df.columns.drop('loan_paid_back')
test_df = test_df[train_cols]

Build a Baseline Model (Logistic Regression)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# 1. Separate features (X) and target (y)
X = train_df.drop('loan_paid_back', axis=1)
y = train_df['loan_paid_back']

# 2. Split data to create a validation set
# This lets us check our score before submitting
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Create and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 4. Check your validation score (ROC AUC)
# IMPORTANT: Use .predict_proba() to get probabilities
val_preds = model.predict_proba(X_val)[:, 1]
auc_score = roc_auc_score(y_val, val_preds)

print(f"Validation ROC AUC Score: {auc_score}")

Validation ROC AUC Score: 0.6837895252929485


In [6]:
# 1. Predict on the test data
test_preds = model.predict_proba(test_df)[:, 1]

# 2. Create the submission DataFrame
submission_df = pd.DataFrame({
    'id': test_ids,
    'loan_paid_back': test_preds
})

# 3. Save the file
# This is the file Kaggle will look for when you submit
submission_df.to_csv('submission.csv', index=False)

print("Submission file created!")

Submission file created!
